# DATA Cleaing 



In [5]:
!wget https://raw.githubusercontent.com/bill-from-ri/JEBS-data/main/training.json


--2025-12-06 12:31:55--  https://raw.githubusercontent.com/bill-from-ri/JEBS-data/main/training.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 651059 (636K) [text/plain]
Saving to: ‘training.json’

training.json       100%[===================>] 635.80K  --.-KB/s    in 0.04s   

2025-12-06 12:31:55 (14.7 MB/s) - ‘training.json’ saved [651059/651059]



In [6]:
import json

with open("/kaggle/working/training.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(type(data))
print(len(data))

<class 'dict'>
100


In [7]:
data.keys()


dict_keys(['Q1_A1', 'Q1_A10', 'Q1_A2', 'Q1_A3', 'Q1_A4', 'Q1_A5', 'Q1_A6', 'Q1_A7', 'Q1_A8', 'Q1_A9', 'Q10_A1', 'Q10_A10', 'Q10_A2', 'Q10_A3', 'Q10_A4', 'Q10_A5', 'Q10_A6', 'Q10_A7', 'Q10_A8', 'Q10_A9', 'Q11_A1', 'Q11_A10', 'Q11_A2', 'Q11_A3', 'Q11_A4', 'Q11_A5', 'Q11_A6', 'Q11_A7', 'Q11_A8', 'Q11_A9', 'Q12_A1', 'Q12_A10', 'Q12_A2', 'Q12_A3', 'Q12_A4', 'Q12_A5', 'Q12_A6', 'Q12_A7', 'Q12_A8', 'Q12_A9', 'Q13_A1', 'Q13_A10', 'Q13_A2', 'Q13_A3', 'Q13_A4', 'Q13_A5', 'Q13_A6', 'Q13_A7', 'Q13_A8', 'Q13_A9', 'Q14_A1', 'Q14_A10', 'Q14_A2', 'Q14_A3', 'Q14_A4', 'Q14_A5', 'Q14_A6', 'Q14_A7', 'Q14_A8', 'Q14_A9', 'Q15_A1', 'Q15_A10', 'Q15_A2', 'Q15_A3', 'Q15_A4', 'Q15_A5', 'Q15_A6', 'Q15_A7', 'Q15_A8', 'Q15_A9', 'Q16_A1', 'Q16_A10', 'Q16_A2', 'Q16_A3', 'Q16_A4', 'Q16_A5', 'Q16_A6', 'Q16_A7', 'Q16_A8', 'Q16_A9', 'Q17_A1', 'Q17_A10', 'Q17_A2', 'Q17_A3', 'Q17_A4', 'Q17_A5', 'Q17_A6', 'Q17_A7', 'Q17_A8', 'Q17_A9', 'Q18_A1', 'Q18_A10', 'Q18_A2', 'Q18_A3', 'Q18_A4', 'Q18_A5', 'Q18_A6', 'Q18_A7', 'Q18_A8',

In [8]:
import json

first_key = list(data.keys())[0]
print(first_key)
print(json.dumps(data[first_key], indent=2))


Q1_A1
{
  "retinal detachment": [
    [
      "EXPLAIN",
      "the damage to the eye that happens when the retina (a light-sensitive layer  at the back of the eye) is pulled away from its normal position"
    ],
    [
      "EXPLAIN",
      "come apart from the wall of the eye"
    ],
    [
      "EXPLAIN",
      "It can come apart from the wall of the eye"
    ],
    [
      "EXPLAIN",
      "damage to the eye that happens when the retina (a light-sensitive layer  at the back of the eye) is pulled away from its normal position"
    ]
  ],
  "visual acuity": [
    [
      "EXPLAIN",
      "ability to see small details in a standard vision test"
    ],
    [
      "SUBSTITUTE",
      "Visual sharpness"
    ],
    [
      "SUBSTITUTE",
      "clarity of many of the patients\u2019 eyesight"
    ],
    [
      "SUBSTITUTE",
      "vision"
    ],
    [
      "GENERALIZE",
      "this decrease"
    ]
  ],
  "cataract": [
    [
      "EXPLAIN",
      "a clouding of the lens in the eye"
    ]

In [9]:
# Run in Kaggle (assumes `data` variable already contains the loaded JSON dict)
import json, re, unicodedata, os
from collections import defaultdict

# ---------- sanitize function ----------
def clean_text(text, lower=False):
    if text is None:
        return ""
    # ensure str
    s = str(text)
    # normalize unicode (decompose + compatibility)
    s = unicodedata.normalize("NFKC", s)
    # replace common smart quotes with ascii quotes
    s = s.replace("\u2019", "'").replace("\u2018", "'").replace("\u201c", '"').replace("\u201d", '"')
    # remove weird control characters
    s = "".join(ch for ch in s if unicodedata.category(ch)[0] != "C")
    # collapse whitespace
    s = re.sub(r"\s+", " ", s).strip()
    # trim leading/trailing punctuation noise
    s = re.sub(r"^[\-\–\—\:\;]+", "", s).strip()
    s = re.sub(r"[\-\–\—\:\;]+$", "", s).strip()
    # optional: lowercasing (set to True if you prefer lowercased dataset)
    if lower:
        s = s.lower()
    return s

# ---------- conversion ----------
pairs_set = set()        # (medical, layman) unique pairs
pairs_by_med = defaultdict(set)  # med -> set(layman) (optional grouping)

count_total_entries = 0
count_explain_kept = 0

for top_key, term_dict in data.items():
    if not isinstance(term_dict, dict):
        continue
    for term, entries in term_dict.items():
        # entries looks like: [ ["EXPLAIN", "text"], ["SUBSTITUTE","..."], ... ]
        if not isinstance(entries, list):
            continue
        for entry in entries:
            count_total_entries += 1
            # defensive unpack
            if not (isinstance(entry, (list,tuple)) and len(entry) >= 2):
                continue
            typ = str(entry[0]).strip().upper()
            txt = entry[1]
            if typ != "EXPLAIN":
                continue
            txt_clean = clean_text(txt, lower=False)
            med_clean = clean_text(term, lower=False)
            # filter empties / tiny
            if len(txt_clean) < 5 or len(med_clean) < 1:
                continue
            pairs_set.add((med_clean, txt_clean))
            pairs_by_med[med_clean].add(txt_clean)
            count_explain_kept += 1

# ---------- produce lists ----------
cleaned_pairs = [{"medical": m, "layman": l} for (m,l) in sorted(pairs_set)]
# optionally, sort by medical then layman
cleaned_pairs.sort(key=lambda x: (x["medical"].lower(), x["layman"].lower()))

# SFT chat-style messages
sft_data = []
for p in cleaned_pairs:
    sft_data.append({
        "messages": [
            {"role": "user", "content": f"Explain this medical term in simple language: {p['medical']}"},
            {"role": "assistant", "content": p["layman"]}
        ]
    })

# ---------- basic filtering / stats ----------
# remove extremely long duplicates if needed (example)
cleaned_pairs = [p for p in cleaned_pairs if len(p["layman"]) < 1000]

print("Total raw entries scanned:", count_total_entries)
print("EXPLAIN entries kept:", count_explain_kept)
print("Unique (medical, layman) pairs:", len(cleaned_pairs))
print("Unique medical terms (approx):", len(pairs_by_med))
print()

# show a few samples
for i, p in enumerate(cleaned_pairs[:8]):
    print(i+1, "MED:", p["medical"], " => LAY:", p["layman"])

# ---------- save files ----------
os.makedirs("/kaggle/working", exist_ok=True)
with open("medical_layman_clean.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_pairs, f, ensure_ascii=False, indent=2)

with open("medical_layman_sft.json", "w", encoding="utf-8") as f:
    json.dump(sft_data, f, ensure_ascii=False, indent=2)

# copy to Kaggle working output area (makes them downloadable)
import shutil
shutil.copy("medical_layman_clean.json", "_layman_clean.json")
shutil.copy("medical_layman_sft.json", "sft.json")

print("\nSaved:")
print(" - medical_layman_clean.json -> /kaggle/working/")
print(" - medical_layman_sft.json -> /kaggle/working/")
print("\nNow go to Kaggle UI: 'Output files' -> download the JSON(s). Then upload to Google Drive.")


Total raw entries scanned: 5392
EXPLAIN entries kept: 1306
Unique (medical, layman) pairs: 1124
Unique medical terms (approx): 534

1 MED: 1-year mortality rate  => LAY: deaths
2 MED: 1/Pcr  => LAY: creatinine clearance divided by serum creatinine concentration
3 MED: 3-beta-hydroxybutyrate  => LAY: blood ketones
4 MED: a bulge in the fovea  => LAY: cuplike depression in the eye
5 MED: a progressive disease  => LAY: disease that keeps getting worse
6 MED: AAA  => LAY: a bulge in a blood vessel caused by a weakness in the blood vessel wall
7 MED: AAA  => LAY: abnormal ballooning of the major abdominal artery
8 MED: AAA  => LAY: an abnormal ballooning of the major abdominal artery

Saved:
 - medical_layman_clean.json -> /kaggle/working/
 - medical_layman_sft.json -> /kaggle/working/

Now go to Kaggle UI: 'Output files' -> download the JSON(s). Then upload to Google Drive.
